In [1]:
from getpass import getpass
from dotenv import load_dotenv
import pandas as pd
import requests
import os
import folium
import pandas as pd

In [2]:
def foursquare_access():
    load_dotenv()
    token = os.getenv("token")
    return token

In [3]:
def category_from_foursquare (lat, lon, category):

    url = f"https://api.foursquare.com/v3/places/search?&ll={lon}%2C{lat}&categories={category}&sort=DISTANCE&limit=3"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [4]:
def query_from_foursquare (query, lat, lon):
    
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lon}%2C{lat}&sort=DISTANCE&limit=3"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [5]:
def everything_from_foursquare (one_element, data):

    name = one_element["name"]
    lat = one_element["geocodes"]["main"]["latitude"]
    lon = one_element["geocodes"]["main"]["longitude"]
    distance = one_element["distance"]
    
    dict_ = {"name": name, "latitude": lat, "longitude": lon, "data": data, "distance": distance}
    
    return dict_

In [6]:
def df_foursquare(df, lat, lon):
    
    category_dict={"Airport": "19040", "Preschool":"12056", "Primary and Secondary School":"12057", "Train Station":"19046",
                   "Night Clubs":"10032", "Vegan Restaurant":"13377", "Basketball Stadium":"18008"}
    
    query_list=["Starbucks","Dog Grooming"]
    
        
    for key, value in category_dict.items():
        response=category_from_foursquare(lon, lat, value)
        new_list = []
        for i in response["results"]:
            new_list.append(everything_from_foursquare (i, key))

        df1 = pd.DataFrame(new_list)
        df = pd.concat([df, df1])
                
                
    for query in query_list:
        response=query_from_foursquare (query,lon, lat)
        new_list = []
        for i in response["results"]:
            new_list.append(everything_from_foursquare (i, query))

        df1 = pd.DataFrame(new_list)
        df = pd.concat([df, df1])
    
    return df

In [7]:
token = foursquare_access()

In [9]:
df_sf = pd.read_csv("../dataframes/df_sanfrancisco.csv")
df_sf = df_foursquare(df_sf, "37.7968917","-122.4032027") 
df_sf.to_csv("../dataframes/df_sanfrancisco.csv", index=False)

df_lon = pd.read_csv("../dataframes/df_london.csv")
df_lon = df_foursquare(df_lon, "51.513248",'-0.154738')
df_lon.to_csv("../dataframes/df_london.csv", index=False)

df_nyc = pd.read_csv("../dataframes/df_newyork.csv")
df_nyc = df_foursquare(df_nyc, '40.7398872','-73.9850319') 
df_nyc.to_csv("../dataframes/df_newyork.csv", index=False)


In [10]:
df_sf

,name,latitude,longitude,data,distance
0,Digg,37.764726,-122.394523,Design company,NaN
1,Wix,37.756720,-122.405070,Design company,NaN
2,99designs,37.795531,-122.400598,Design company,NaN
3,Minted,37.797435,-122.403175,Design company,NaN
4,Netbiscuits,37.755037,-122.426442,Design company,NaN
5,Oakland International Airport (OAK),37.712395,-122.213323,Airport,19161.0
6,San Francisco International Airport (SFO),37.616581,-122.386689,Airport,20067.0
7,Imane's Wonderschool,37.780740,-122.407515,Preschool,1834.0
8,Presidio Knolls School,37.773203,-122.413809,Preschool,2783.0
9,Alta Plaza Preschool,37.790221,-122.437322,Preschool,3138.0


In [11]:
df_lon

,name,latitude,longitude,data,distance
0,Netbiscuits,51.513248,-0.154738,Design company,NaN
1,Moonfruit,51.519130,-0.110160,Design company,NaN
2,London City Airport (LCY),51.504029,0.049552,Airport,14300.0
3,London Heathrow Airport (LHR),51.470584,-0.454958,Airport,21466.0
4,Home Star Childcare,51.513242,-0.141109,Preschool,1075.0
5,Dyason Preschool,51.489346,-0.088069,Preschool,5446.0
6,Turning Earth Ceramics,51.534412,-0.075928,Preschool,6069.0
7,Netley Primary School,51.527677,-0.140121,Primary and Secondary School,1971.0
8,Eaton House School,51.494096,-0.154935,Primary and Secondary School,2153.0
9,Westminster Academy,51.521012,-0.192724,Primary and Secondary School,2583.0


In [12]:
df_nyc

,name,latitude,longitude,data,distance
0,Gilt Groupe,40.747270,-73.980064,Design company,NaN
1,Squarespace,40.720810,-74.000870,Design company,NaN
2,Banyan Branch,40.695170,-73.932300,Design company,NaN
3,Netbiscuits,40.695170,-73.932300,Design company,NaN
0,LaGuardia Airport (LGA),40.774527,-73.871941,Airport,10447.0
1,Newark Liberty International Airport (EWR),40.690377,-74.177145,Airport,17068.0
2,John F. Kennedy International Airport (JFK),40.643864,-73.782268,Airport,19901.0
0,Metrokids Preschool,40.748664,-73.970435,Preschool,1563.0
1,Academy of St Joseph,40.732732,-74.000991,Preschool,1571.0
2,FIAF Preschool,40.763926,-73.970723,Preschool,2930.0
